In [1]:
import os
import pandas as pd
import numpy as np
import re
from pandarallel import pandarallel
import concurrent.futures


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
cov_folder = '/home/laura/ANALYSIS/VARIANT_CALLING/mezclas_sin_captura/23_11_18_MEZCLAS_SIN_CAPTURA/Coverages'
cov_folder_test="/home/laura/ANALYSIS/VARIANT_CALLING/mezclas_sin_captura/test_cov_parallel"

In [3]:
def calculate_cov_stats_L(file_cov):
    sample = file_cov.split("/")[-1].split(".")[0]
    df = pd.read_csv(file_cov, sep="\t", names=["#CHROM", "POS", "COV"])
    unmmaped_pos = len(df.POS[df.COV == 0].tolist())
    pos_0_10 = len(df.POS[(df.COV > 0) & (df.COV <= 10)].tolist())
    pos_10_20 = len(df.POS[(df.COV > 10) & (df.COV <= 20)].tolist())
    pos_high20 = len(df.POS[(df.COV > 20)].tolist())
    pos_high50 = len(df.POS[(df.COV > 50)].tolist())
    pos_high100 = len(df.POS[(df.COV >= 100)].tolist())
    pos_high500 = len(df.POS[(df.COV >= 500)].tolist())
    pos_high1000 = len(df.POS[(df.COV >= 1000)].tolist())
    total_pos = df.shape[0]
    unmmaped_prop = "%.2f" % ((unmmaped_pos/total_pos)*100)
    prop_0_10 = "%.2f" % ((pos_0_10/total_pos)*100)
    prop_10_20 = "%.2f" % ((pos_10_20/total_pos)*100)
    prop_high20 = "%.2f" % ((pos_high20/total_pos)*100)
    prop_high50 = "%.2f" % ((pos_high50/total_pos)*100)
    prop_high100 = "%.2f" % ((pos_high100/total_pos)*100)
    prop_high500 = "%.2f" % ((pos_high500/total_pos)*100)
    prop_high1000 = "%.2f" % ((pos_high1000/total_pos)*100)

    mean_cov = "%.2f" % (df.COV.mean())

    return sample,mean_cov, unmmaped_prop, prop_0_10, prop_10_20, prop_high20, prop_high50, prop_high100, prop_high500, prop_high1000

In [19]:
def calculate_cov_stats(file_cov):
    sample = file_cov.split("/")[-1].split(".")[0]
    df = pd.read_csv(file_cov, sep="\t", names=["#CHROM", "POS", "COV"])
    unmmaped_pos = len(df.POS[df.COV == 0].tolist())
    pos_0_10 = len(df.POS[df.COV >= 5].tolist())
    pos_10_20 = len(df.POS[df.COV >= 20].tolist())
    pos_high20 = len(df.POS[(df.COV >= 20)].tolist())
    pos_high50 = len(df.POS[(df.COV >= 50)].tolist())
    pos_high100 = len(df.POS[(df.COV >= 100)].tolist())
    pos_high500 = len(df.POS[(df.COV >= 500)].tolist())
    pos_high1000 = len(df.POS[(df.COV >= 1000)].tolist())
    total_pos = df.shape[0]
    unmmaped_prop = "%.2f" % ((unmmaped_pos/total_pos)*100)
    prop_0_10 = "%.2f" % ((pos_0_10/total_pos)*100)
    prop_10_20 = "%.2f" % ((pos_10_20/total_pos)*100)
    prop_high20 = "%.2f" % ((pos_high20/total_pos)*100)
    prop_high50 = "%.2f" % ((pos_high50/total_pos)*100)
    prop_high100 = "%.2f" % ((pos_high100/total_pos)*100)
    prop_high500 = "%.2f" % ((pos_high500/total_pos)*100)
    prop_high1000 = "%.2f" % ((pos_high1000/total_pos)*100)

    mean_cov = "%.2f" % (df.COV.mean())

    return sample,mean_cov, unmmaped_prop, prop_0_10, prop_10_20, prop_high20, prop_high50, prop_high100, prop_high500, prop_high1000


In [20]:
calculate_cov_stats('/home/laura/ANALYSIS/VARIANT_CALLING/mezclas_sin_captura/23_11_18_MEZCLAS_SIN_CAPTURA/Coverages/8_S2_L001_.merged.bam.cov')

('8_S2_L001_',
 '0.87',
 '53.30',
 '1.10',
 '0.04',
 '0.04',
 '0.02',
 '0.00',
 '0.00',
 '0.00')

In [21]:
def obtain_group_cov_stats(directory, group_name):
    directory_path = os.path.abspath(directory)

    output_group_name = group_name + ".coverage.summary.tab"
    output_file = os.path.join(directory_path, output_group_name)

    columns=["#SAMPLE","MEAN_COV","UNMMAPED_PROP","COV1-10X","COV10-20X","COV>20X","COV>50X","COV>100X" ,"COV>500X","COV>1000X"]

    

    files_list = []

    for root, _, files in os.walk(directory):
        if root == directory: 
            for name in files:
                if name.endswith('.cov'):
                    filename = os.path.join(root, name)
                    #df[columns] = df.apply(calculate_cov_stats(filename), axis=1, result_type="expand")
                    files_list.append(filename)

    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        dfs = executor.map(calculate_cov_stats, files_list)
    df = pd.DataFrame(dfs, columns=columns)
    
    df.to_csv(output_file, sep="\t", index=False)
    
    return df

In [22]:
%%time
obtain_group_cov_stats(cov_folder_test, 'group')

CPU times: user 29.8 s, sys: 26.6 s, total: 56.4 s
Wall time: 5.86 s


,#SAMPLE,MEAN_COV,UNMMAPED_PROP,COV1-10X,COV10-20X,COV>20X,COV>50X,COV>100X,COV>500X,COV>1000X
0,P14-98629,899.04,0.14,99.59,99.21,99.21,98.79,98.33,94.71,32.14
1,A13908,668.21,0.48,99.46,99.37,99.37,99.16,98.72,89.02,0.75
2,A12157,835.49,0.50,99.46,99.38,99.38,99.20,98.88,94.37,14.68


In [24]:
%%time
df = obtain_group_cov_stats(cov_folder, 'group')

CPU times: user 1min 27s, sys: 1min 33s, total: 3min 1s
Wall time: 11.3 s


In [25]:
df

,#SAMPLE,MEAN_COV,UNMMAPED_PROP,COV1-10X,COV10-20X,COV>20X,COV>50X,COV>100X,COV>500X,COV>1000X
0,8_S2_L001_,0.87,53.30,1.10,0.04,0.04,0.02,0.00,0.00,0.00
1,1_S4_L001_,1.13,45.09,2.34,0.04,0.04,0.03,0.00,0.00,0.00
2,5_S7_L001_,1.33,39.83,3.40,0.05,0.05,0.03,0.02,0.00,0.00
3,7_S8_L001_,0.88,55.62,1.34,0.05,0.05,0.03,0.00,0.00,0.00
4,12_S11_L001_,1.11,46.25,2.17,0.04,0.04,0.03,0.00,0.00,0.00
5,15_S12_L001_,1.26,40.40,2.88,0.04,0.04,0.02,0.00,0.00,0.00
6,13_S3_L001_,0.92,51.00,1.22,0.04,0.04,0.02,0.00,0.00,0.00
7,2_S5_L001_,1.26,43.64,3.31,0.04,0.04,0.03,0.00,0.00,0.00
8,10_S9_L001_,1.36,40.51,3.73,0.05,0.05,0.03,0.01,0.00,0.00
9,11_S10_L001_,1.08,48.11,2.09,0.05,0.05,0.03,0.00,0.00,0.00


In [27]:
df.to_csv("/home/laura/ANALYSIS/VARIANT_CALLING/mezclas_sin_captura/summary_coverage.tsv", sep="\t", index=False)